# Проект по SQL

   Коронавирус застал мир врасплох, изменив привычный порядок вещей. В свободное время жители городов больше не выходят на улицу, не посещают кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать.\
   Ваша компания решила быть на волне и купила крупный сервис для чтения книг по подписке. Ваша первая задача как аналитика — проанализировать базу данных. В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine

In [2]:
# устанавливаем параметры
db_config = {'user':'praktikum_student', # имя пользователя
'pwd':'Sdf4$2;d-d30pp', # пароль
'host':'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # портподключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config) 
# сохраняем коннектор
engine = create_engine(connection_string,connect_args={'sslmode':'require'})

In [3]:
# чтобы выполнить SQL-запрос, используем Pandas
query = '''SELECT * FROM books LIMIT 5'''

con = engine.connect()

pd.io.sql.read_sql(sql=text(query),con=con)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [4]:
additional_task = '''


SELECT DATE_TRUNC('year', CAST(publication_date AS timestamp)) AS year_publication,
       COUNT(DISTINCT (publisher_id)) AS publisher_cnt,
       COUNT(book_id) AS book_cnt,
       SUM(num_pages)/1000.0 AS num_pages_thousands
FROM books
GROUP BY year_publication
HAVING COUNT(book_id) > 30



'''

pd.io.sql.read_sql(sql=text(additional_task),con=con)

,year_publication,publisher_cnt,book_cnt,num_pages_thousands
0,1999-01-01,26,41,15.763
1,2000-01-01,35,38,13.328
2,2001-01-01,41,60,21.758
3,2002-01-01,62,94,38.597
4,2003-01-01,65,105,41.423
5,2004-01-01,88,124,46.779
6,2005-01-01,89,139,55.967
7,2006-01-01,109,184,68.302
8,2007-01-01,38,50,18.258


## Посчитаем, сколько книг вышло после 1 января 2000 года

In [5]:
task1 = '''
SELECT COUNT(book_id) AS book_cnt
FROM books
WHERE publication_date > '2000-01-01';
'''
pd.io.sql.read_sql(sql=text(task1),con=con)

,book_cnt
0,819


После 1 января 2000 года вышло 819 книг

## Для каждой книги посчитаем количество обзоров и среднюю оценку

In [6]:
task2 = ''' 

WITH

jt1 AS (SELECT book_id, AVG(rating) AS rating_avg
        FROM ratings
        GROUP BY book_id),
        
jt2 AS (SELECT book_id, COUNT(review_id) AS review_cnt
        FROM reviews
        GROUP BY book_id)

SELECT b.title AS title, jt2.review_cnt AS reviews, jt1.rating_avg AS avg_rating
FROM books AS b LEFT JOIN jt1 ON jt1.book_id = b.book_id 
                LEFT JOIN jt2 ON jt2.book_id = b.book_id;
                 
''' 
 

pd.io.sql.read_sql(sql=text(task2), con = engine)

,title,reviews,avg_rating
0,The Body in the Library (Miss Marple #3),2.0,4.500000
1,Galápagos,2.0,4.500000
2,A Tree Grows in Brooklyn,5.0,4.250000
3,Undaunted Courage: The Pioneering First Missio...,2.0,4.000000
4,The Prophet,4.0,4.285714
...,...,...,...
995,Alice in Wonderland,4.0,4.230769
996,A Woman of Substance (Emma Harte Saga #1),2.0,5.000000
997,Christine,3.0,3.428571
998,The Magicians' Guild (Black Magician Trilogy #1),2.0,3.500000


Для каждой книги из базы рассчитали количество отзывов и среднюю оценку.

## Определим издательство, которое выпустило наибольшее число книг толще 50 страниц — так мы исключим из анализа брошюры

In [7]:
task3 = '''

SELECT p.publisher, COUNT(b.book_id) AS book_cnt
FROM books AS b LEFT JOIN publishers AS p ON b.publisher_id = p.publisher_id
WHERE num_pages > 50
GROUP BY publisher
ORDER BY book_cnt DESC
LIMIT 1;

'''
pd.io.sql.read_sql(sql=text(task3), con = engine)

,publisher,book_cnt
0,Penguin Books,42


Издательство, которое выпустило наибольшее число книг толще 50 страниц - "Penguin Books" (42 книги).

## Определим автора с самой высокой средней оценкой книг — учитывать будем только книги с 50 и более оценками

In [8]:
task4 = '''

WITH

jt AS (SELECT b.book_id, a.author, AVG(rating) AS avg_rating
       FROM books b LEFT JOIN authors a ON b.author_id = a.author_id
                    LEFT JOIN ratings r ON b.book_id = r.book_id
       GROUP BY b.book_id, a.author_id
       HAVING COUNT(rating_id) >=50)
                   
SELECT author, AVG(jt.avg_rating) AS total_rating
FROM jt
GROUP BY jt.author
ORDER BY total_rating DESC
LIMIT 1;

'''
pd.io.sql.read_sql(sql=text(task4), con = engine)

,author,total_rating
0,J.K. Rowling/Mary GrandPré,4.283844


Автор с самой высокой средней оценкой книг среди книг с 50 и более оценками - J.K. Rowling/Mary GrandPré.

## Посчитаем среднее количество обзоров от пользователей, которые поставили больше 48 оценок

In [9]:
task5 = '''

SELECT AVG(review_cnt) AS review_avg
FROM (SELECT username, COUNT(review_id) AS review_cnt
      FROM reviews
      WHERE username IN (SELECT username
                         FROM ratings
                         GROUP BY username
                         HAVING COUNT(rating_id) > 48)
      GROUP BY username) AS user48;

'''
pd.io.sql.read_sql(sql=text(task5), con = engine)

,review_avg
0,24.0


Среднее количество обзоров от пользователей, которые поставили больше 48 оценок составляет 24.

**ВЫВОД:** 
- После 1 января 2000 года вышло 819 книг
- Для каждой книги из базы были рассчитаны количество отзывов и средняя оценка
- Издательство, выпустившее наибольшее число книг толще 50 страниц - "Penguin Books" (42 книги)
- Автор с самой высокой средней оценкой книг среди книг с 50 и более оценками - J.K. Rowling/Mary GrandPré
- Среднее количество обзоров от пользователей, которые поставили больше 48 оценок составляет 24

Можно рекомендовать активнее продвигать книги J.K. Rowling.